In [151]:
import pandas as pd
import numpy as np

df = pd.read_csv('CandidateApplicationTrackerWithoutLeadIDs.csv',index_col=0)

In [152]:
print(df.columns)

Index(['Candidate Name', 'Counselor Name', 'Block Amount Received Date',
       'Univ. Pitched', 'Candidate Preferred College #1',
       'Candidate Preferred College #2', '10th Percentage ',
       '12th Percentange', 'UG - CGPA (out of 10) ', 'GPA (Out of 4)',
       'PG - CGPA or %', 'Work Exp.', 'Highest Education',
       'Designation - Company Name ', 'Candidate Application Status',
       'Candidate Loan Status', 'KYC & Acad Documents Received',
       'Tech Exam Scores', 'Coding Test Score', 'English Exam Scores',
       'English Interview Score (15)', 'Tech. interview Score (10)',
       'Application Type', 'Intake Status', 'U1Choice of School #1',
       'U1Choice of Program #1', 'U1Passport #1', 'U1Resume #1', 'U1Degree #1',
       'U1Transcript #1', 'U1Application Status #1', 'U1Offer Letter #1',
       'U2Choice of School #2', 'U2Choice of Program #2',
       'U2Application Status #2', 'U2Offer Letter #2', 'U2Deposit #2',
       'U3Choice of School #3', 'U3Choice of Progra

Removing extra bits of strings from data and ready it for conversion to float

In [153]:
import re
misc = ['10th Percentage ','12th Percentange','GPA (Out of 4)']
scoresCol = [ 'UG - CGPA (out of 10) ', 'PG - CGPA or %','10th Percentage ','12th Percentange','GPA (Out of 4)']

def remove_section(val, section):
    if pd.isnull(val):
        return val
    val = str(val)
    return val.replace(section, '').strip()



def remove_brackets(val):
    if pd.isnull(val):
        return val
    val = str(val)
    result = ''
    skip = 0
    for char in val:
        if char == '(':
            skip += 1
        elif char == ')' and skip > 0:
            skip -= 1
        elif skip == 0:
            result += char
    return result.strip()

sections=[' gpa',' cgpa',' - diploma in civil engg',' - diploma',' credits','%',' cpi',' credits']

for section_str in sections:
    for col in scoresCol:
        df[col] = df[col].astype(str)
        df[col] = df[col].apply(lambda x: remove_section(x, section_str))
        df[col] = df[col].apply(remove_brackets)
    

In [154]:
sections=[' gpa',' cgpa',' - diploma in civil engg',' - diploma',' credits','%',' cpi',' credits']

def replace_section(val, section, r):
    if pd.isnull(val):
        return val
    val = str(val)
    return val.replace(section, r).strip()

for col in scoresCol:
    df[col] = df[col].apply(lambda x: replace_section(x, 'a+','95' ))
    df[col] = df[col].apply(lambda x: replace_section(x, 'na','0' ))
    df[col] = df[col].apply(lambda x: replace_section(x, 'f','0' ))

In [155]:
def process_value(val):
    if pd.isnull(val):
        return val
    if isinstance(val, str) and '/4' in val:
        val = val.replace('/4', '')
        try:
            result = float(val) * 2.5
            return str(result)
        except ValueError:
            return val
    return val

for col in scoresCol:
    df[col] = df[col].apply(process_value)

In [156]:
for col in scoresCol:
    df[col] = df[col].replace('-', '')
    df[col] = df[col].replace('', '0')

Converting to float

In [157]:
for col in scoresCol:
        df[col] = df[col].astype(float)

Changing all floats to be out of 100

In [158]:
outoften = ['UG - CGPA (out of 10) ', 'PG - CGPA or %']

for i in range(3):
    for col in outoften:
        df[col] = df[col].apply(lambda x : x*10 if x<1 else x)
        df[col] = df[col].apply(lambda x : x/10 if x>10 else x)

for i in range(3):
    df['GPA (Out of 4)'] = df['GPA (Out of 4)'].apply(lambda x : x/10 if x>4 else x)

In [159]:
#DO NOT RUN MORE THAN ONCE
for col in outoften:
    df[col] = df[col].apply(lambda x : x*10)

df['GPA (Out of 4)'] = df['GPA (Out of 4)'].apply(lambda x : x*25)

Cleaning Experience years

In [160]:
sections=['years','year','+']

for section_str in sections:
    df['Work Exp.'] = df['Work Exp.'].astype(str)
    df['Work Exp.'] = df['Work Exp.'].apply(lambda x: remove_section(x, section_str))

In [161]:
df['Work Exp.'] = df['Work Exp.'].apply(lambda x: x.replace(' ', ''))

In [162]:
def range_to_average(val):
    if pd.isnull(val):
        return val
    if isinstance(val, str) and '-' in val:
        try:
            start, end = val.split('-')
            start, end = float(start), float(end)
            average = (start + end) / 2
            return average
        except ValueError:
            return val
    return val

df['Work Exp.'] = df['Work Exp.'].apply(range_to_average)

In [163]:
df['Work Exp.'] = df['Work Exp.'].astype(str)
df['Work Exp.'] = df['Work Exp.'].apply(lambda x: x.replace('7months', '0.6'))
df['Work Exp.'] = df['Work Exp.'].apply(lambda x: x.replace('fresher', '0'))

In [164]:
df['Work Exp.'] = df['Work Exp.'].astype(float)

Saving into output.xlsx

In [165]:
df.to_excel('output.xlsx')